In [2]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [3]:
from inference_auth_token import get_access_token
from langchain_openai import ChatOpenAI

access_token = get_access_token()
base_url="https://inference-api.alcf.anl.gov/resource_server/sophia/vllm/v1"
model="Qwen/Qwen3-32B"
try:
    llm = ChatOpenAI(model=model, base_url=base_url, api_key=access_token)
    print(f"Successfully loaded model: {model} from {base_url}")

except Exception as e:
    print(f"Error with loading {model}")
    print(e)

Successfully loaded model: Qwen/Qwen3-32B from https://inference-api.alcf.anl.gov/resource_server/sophia/vllm/v1


In [4]:
# Using pre-built ReAct agent from LangGraph example: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
from typing import Literal
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")

# Define the tools and graph

tools = [get_weather]
weather_graph = create_react_agent(llm, tools=tools)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
            
inputs = {"messages": [("user", "what is the weather in sf")],}
print_stream(weather_graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

what is the weather in sf
================================== Ai Message ==================================
Tool Calls:
  get_weather (chatcmpl-tool-daa23a1373c842de81abeab591de9445)
 Call ID: chatcmpl-tool-daa23a1373c842de81abeab591de9445
  Args:
    city: sf
================================= Tool Message =================================
Name: get_weather

It's always sunny in sf
================================== Ai Message ==================================



The weather in San Francisco, according to the latest report: **"It's always sunny in sf."** ☀️ 

(If only real-life weather were this reliable!) Let me know if you need anything else!


In [5]:
%%capture --no-stderr
%pip install pubchempy

In [6]:
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent

@tool
def molecule_name_to_smiles(name: str) -> str:
    """Convert a molecule name SMILES format.

    Args:
        name (str): molecule name.

    Returns:
        str: SMILES string.
    """
    import pubchempy
    return pubchempy.get_compounds(str(name), "name")[0].canonical_smiles

tools = [molecule_name_to_smiles]
chem_graph = create_react_agent(llm, tools=tools)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
            
inputs = {"messages": [("user", "what is SMILES string of Carbon Dioxide?")]}
print_stream(chem_graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

what is SMILES string of Carbon Dioxide?
================================== Ai Message ==================================
Tool Calls:
  molecule_name_to_smiles (chatcmpl-tool-aee89a7716124419943c34e0fea40883)
 Call ID: chatcmpl-tool-aee89a7716124419943c34e0fea40883
  Args:
    name: Carbon Dioxide


/var/folders/f9/ff27tdm11x91185x3q1v6_lm0000gq/T/ipykernel_26776/3463582542.py:15: PubChemPyDeprecationWarning: canonical_smiles is deprecated: Use connectivity_smiles instead
  return pubchempy.get_compounds(str(name), "name")[0].canonical_smiles


================================= Tool Message =================================
Name: molecule_name_to_smiles

C(=O)=O
================================== Ai Message ==================================



The SMILES string for Carbon Dioxide (CO₂) is **C(=O)=O**. 

This notation represents a central carbon atom double-bonded to two oxygen atoms, reflecting its linear molecular structure. Let me know if you need further clarification!


In [7]:
inputs = {"messages": [("user", "what is SMILES string of 1-ethyl-3-(1-naphthalen-2-ylethenylamino)thiourea?")]}
print_stream(chem_graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

what is SMILES string of 1-ethyl-3-(1-naphthalen-2-ylethenylamino)thiourea?
================================== Ai Message ==================================
Tool Calls:
  molecule_name_to_smiles (chatcmpl-tool-bd5b394356b141f69af6763b83de6f1f)
 Call ID: chatcmpl-tool-bd5b394356b141f69af6763b83de6f1f
  Args:
    name: 1-ethyl-3-(1-naphthalen-2-ylethenylamino)thiourea


/var/folders/f9/ff27tdm11x91185x3q1v6_lm0000gq/T/ipykernel_26776/3463582542.py:15: PubChemPyDeprecationWarning: canonical_smiles is deprecated: Use connectivity_smiles instead
  return pubchempy.get_compounds(str(name), "name")[0].canonical_smiles


================================= Tool Message =================================
Name: molecule_name_to_smiles

CCNC(=S)NNC(=C)C1=CC2=CC=CC=C2C=C1
================================== Ai Message ==================================



The SMILES string for **1-ethyl-3-(1-naphthalen-2-ylethenylamino)thiourea** is:

`CCNC(=S)NNC(=C)C1=CC2=CC=CC=C2C=C1`

This represents the molecular structure with:
- An **ethyl group** on the first nitrogen of the thiourea core,
- A **naphthalene ring** (fused benzene rings) connected via an ethenyl (CH₂=CH−) group to the second nitrogen,
- The thiourea linkage (−NH−C(=S)−NH−). 

Let me know if you need further clarification!
